In [1]:
# from google.colab import drive

# drive.mount('/content/drive', force_remount=True)

In [2]:
import os

In [3]:
# root_dir = '/content/drive/My Drive/09_research_main/lab_01/data/dataP/test'
root_root = '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/'
root_dir = '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data'

In [4]:
def checkpath(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [27]:
new_DATA_PATH =  os.path.join(root_dir, '../dataset_C_v_3.0.1')
checkpath(new_DATA_PATH)

In [6]:
# !pip install labelme

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import json
from labelme.utils.shape import labelme_shapes_to_label

In [9]:
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt

In [10]:
def get_dir_json_list(root_dir):
    jsons_files = []
    for json_N in os.listdir(root_dir):
        if json_N[-1] == 'n':
            jsons_files += [os.path.join(root_dir, json_N)]
    return jsons_files

In [11]:
def get_dict(root_dir, image_DIR):

    json_list = get_dir_json_list(root_dir)

    dict_all = {}
    for json_path in json_list:
        dir,file = os.path.split(json_path)
        file_name = file.split('.')[0]
        image_path = os.path.join(image_DIR, file_name + '.jpg')
        dict_all[image_path] = json_path
    return dict_all

In [12]:
def kakunin_size(size):
  if size==1080:
    x = 420
    y = 0
    w = 1080
    h = 1080
  elif size==488:
    x = 116
    y = 0
    w = 488
    h = 488
  return x, y, w, h

In [13]:
def mask_process(this_DIR, json_DIR, image_DIR, error_list):
    dict_ = get_dict(json_DIR, image_DIR)

    for image_path in dict_:
        mask = []
        class_id = []
        key_ = []

        image = cv2.imread(image_path)
        json_path = dict_[image_path]
        # print('looking at ' + str(image_path))
        # print('looking at ' + str(json_path))
        x, y, w, h = kakunin_size(1080)


        data = json.load(open(json_path))
        try:
            lbl, lbl_names = labelme_shapes_to_label(image.shape, data['shapes'])
        except AssertionError:
            error_list += [json_path]
            print('Got an error in ' + json_path.replace(root_root, ''))
            print('Jumping over!!')
            continue
        except AttributeError:
            error_list += [json_path]
            print('Got an error in ' + json_path.replace(root_root, ''))
            print('Jumping over!!')
            continue

        for i in range(1, len(lbl_names)):  # 跳过第一个class（因为0默认为背景,跳过不取！）
            key = [k for k, v in lbl_names.items() if v == i][0]
            mask.append((lbl == i).astype(np.uint8))  # 举例：当解析出像素值为1，此时对应第一个mask 为0、1组成的（0为背景，1为对象）
            class_id.append(i)  # mask与class_id 对应记录保存
            key_.append(key)
        mask = np.asarray(mask, np.uint8)
        mask = np.transpose(np.asarray(mask, np.uint8), [1, 2, 0])
        image_name = os.path.basename(image_path).split('.')[0]
        dir_ = os.path.dirname(image_path)
        for i in range(0, len(class_id)):
            image_name_ = "{}_mask_{}_{}.jpg".format(image_name,key_[i],i)
            dir_path =  os.path.join(dir_, '../mask',key_[i]) # 构建保存缺陷的文件夹 key_[i]为缺陷名称，i为缺陷ID
            checkpath(dir_path)
            image_path_ = os.path.join(dir_path,image_name_)
            retval, im_at_fixed = cv2.threshold(mask[:,:,i], 0, 255, cv2.THRESH_BINARY)
            
            im_at_fixed = cv2.resize(im_at_fixed, (1920, 1080))

            im_at_fixed = im_at_fixed[y:y+h, x:x+w]

            cv2.imwrite(image_path_, im_at_fixed)

    with open(os.path.join(this_DIR, 'mask/' + json_DIR.split('/')[-1] + '_ERROR_LIST.txt'), 'w') as f:
        for get_error in error_list:
            f.write(str(get_error).replace(root_root, '') + '\n')
    return error_list

In [14]:
def convert_jpg_to_unet(this_dir, image_dir, this_type):
    
  jpgs_DIR = os.path.join(this_dir, 'mask/jpgs')
  checkpath(jpgs_DIR)

  for jpg_N in os.listdir(os.path.join(this_dir, 'mask/frontglottis')):
    if jpg_N[-1] == 'g':
      if (jpg_N.split('.')[0])[-1] != '0':
        print('Got a bad news :(')
      
      jpgs_name = jpg_N.split('_mask_')[0] + '.jpg'

      img = cv2.imread(os.path.join(image_dir, jpgs_name))
      filename = os.path.join(jpgs_DIR, jpgs_name)
      # print('facing ' + filename.replace(root_root, ''))

      img = cv2.resize(img, (1920, 1080))
      
      x, y, w, h = kakunin_size(img.shape[0])

      img = img[y:y+h, x:x+w]

      cv2.imwrite(filename, img)

  checkpath(os.path.join(this_dir, 'mask/'+str(w)))

  checkpath(os.path.join(this_dir, 'mask/'+this_type))

In [15]:
# def convert_jpg_to_unet_pass_GaussianBlur(this_dir, image_dir, this_type):
    
#   jpgs_DIR = os.path.join(this_dir, 'mask/jpgs')
#   checkpath(jpgs_DIR)

#   for jpg_N in os.listdir(os.path.join(this_dir, 'mask/frontglottis')):
#     if jpg_N[-1] == 'g':
#       if (jpg_N.split('.')[0])[-1] != '0':
#         print('Got a bad news :(')
      
#       jpgs_name = jpg_N.split('_mask_')[0] + '.jpg'

#       img = cv2.imread(os.path.join(image_dir, jpgs_name))
#       filename = os.path.join(jpgs_DIR, jpgs_name)
#       # print('facing ' + filename.replace('/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data', ''))

#       img = cv2.resize(img, (1920, 1080))
      
#       x, y, w, h = kakunin_size(img.shape[0])

#       img = img[y:y+h, x:x+w]

#        # kokonichangearu
#       img = cv2.GaussianBlur(img, (9,9), 0.0)

#       cv2.imwrite(filename, img)

#   checkpath(os.path.join(this_dir, 'mask/'+str(w)))

#   checkpath(os.path.join(this_dir, 'mask/'+this_type))

In [16]:
DIR_list = []

for get_folder in os.listdir(root_dir):
  watching_root_dir = os.path.join(root_dir, get_folder)
  for get_folder_name in os.listdir(watching_root_dir):
    watching_this_dir = os.path.join(watching_root_dir, get_folder_name)
    DIR_list += [watching_this_dir]


print(DIR_list)

['/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_8', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_4', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_2', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_5', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_6', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_7', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_9', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_3', '/home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_1']


In [17]:
# main

# data_checker(root_dir)

error_list = []

for root_dir_id in DIR_list:

  this_type = root_dir_id.split('/')[-2]

  print('\nNow processing in ' + root_dir_id.replace(root_root, '') + '\n')

  for get_folder in os.listdir(root_dir_id):
    if get_folder.split('_')[-1] == 'json':
      json_dir = os.path.join(root_dir_id, get_folder)

      for get_jpgs_folder in os.listdir(root_dir_id):
        if get_jpgs_folder.split('_')[-1] == 'jpg':
          image_dir = os.path.join(root_dir_id, get_jpgs_folder)

          error_list += [mask_process(root_dir_id, json_dir, image_dir, error_list)]

          print('\nMask process Completed')

          convert_jpg_to_unet(root_dir_id, image_dir, this_type)

          print('\nProcess Completed in ' + root_dir_id.replace(root_root, '') + '\n')


# print('I found errors in :')
# print(error_list)

with open('ERROR_LIST.txt', 'w') as f:
    for get_error in error_list:
        f.write(str(get_error).replace(root_root, '') + '\n')

print('\n\nSuccessfully Completed!!!\n\n')

labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.



Now processing in data/TEST_TEST/G_8



labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_la


Mask process Completed


labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.


...


Mask process Completed


labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.



Process Completed in data/TEST_TEST/G_3


Now processing in data/TEST_TEST/G_1



labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_label is deprecated, so please use shapes_to_label.
labelme_shapes_to_la


Mask process Completed

Process Completed in data/TEST_TEST/G_1



Successfully Completed!!!




In [18]:
# # main

# # data_checker(root_dir)

# TYPE = 'NORMAL'
# error_list = []

# for get_folder_name in os.listdir(root_dir):
#   watching_this_dir = os.path.join(root_dir, get_folder_name)
#   print('\nNow processing in ' + watching_this_dir + '\n')

#   for get_folder in os.listdir(watching_this_dir):
#     if get_folder[-1] == 'n':
#       processing_dir = os.path.join(watching_this_dir, get_folder)

#       error_list += [mask_process(processing_dir, error_list)]

#       print('\nMask process Completed')

#       convert_jpg_to_unet(processing_dir, TYPE)

#       print('\nProcess Completed in ' + watching_this_dir + '\n')

# print('\n\nSuccessfully Completed!!!\n\n')

# print('I found errors in ' + error_list)


In [19]:
# error_list

In [18]:
# frontglottis

def get_frontglottis(get_dir, get_file):
  frontglottis_DIR = os.path.join(get_dir, 'frontglottis')

  for get_file_name in os.listdir(frontglottis_DIR):
    if get_file_name.replace(get_file_name.split('_')[-1], '') == get_file + '_mask_frontglottis_':
      
      print(os.path.join(frontglottis_DIR.replace(root_dir, ''), get_file_name))
      img = cv2.imread(os.path.join(frontglottis_DIR, get_file_name))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # plt.imshow(img)
      # plt.show()
  return img

In [19]:
# vfleft

def get_vfleft(get_dir, get_file):
  vfleft_DIR = os.path.join(get_dir, 'vfleft')

  for get_file_name in os.listdir(vfleft_DIR):
    if get_file_name.replace(get_file_name.split('_')[-1], '') == get_file + '_mask_vfleft_':
      
      # print(os.path.join(vfleft_DIR, get_file_name))
      img = cv2.imread(os.path.join(vfleft_DIR, get_file_name))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # plt.imshow(img)
      # plt.show()
  return img

In [20]:
# vfright

def get_vfright(get_dir, get_file):
  vfright_DIR = os.path.join(get_dir, 'vfright')

  for get_file_name in os.listdir(vfright_DIR):
    if get_file_name.replace(get_file_name.split('_')[-1], '') == get_file + '_mask_vfright_':

      # print(os.path.join(vfright_DIR, get_file_name))
      img = cv2.imread(os.path.join(vfright_DIR, get_file_name))
      img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      # plt.imshow(img)
      # plt.show()
  return img

In [21]:
# combine

def combine(mask_DIR, get_file, get_TYPE):
  mask_frontglottis = get_frontglottis(mask_DIR, get_file)
  mask_vfleft = get_vfleft(mask_DIR, get_file)
  mask_vfright = get_vfright(mask_DIR, get_file)
  # 0 = ground

  # 1 = frontglottis
  mask_frontglottis[mask_frontglottis > 0] = int(1)

  # 2 = vfright_normal
  # 3 = vfleft_normal
  if get_TYPE == 'NORMAL' or get_TYPE == 'NORMAL_TEST' or get_TYPE == 'NORMAL_VALID':
    mask_vfright[mask_vfright > 0] = int(2)
    mask_vfleft[mask_vfleft > 0] = int(3)
    # print('now is ' + get_TYPE + ' type.')

  # 4 = vfright_rln_R
  # 5 = vfleft_rln_R
  if get_TYPE == 'RLN_R' or get_TYPE == 'RLN_R_TEST' or get_TYPE == 'RLN_R_VALID':
    mask_vfright[mask_vfright > 0] = int(2) # int(4)
    mask_vfleft[mask_vfleft > 0] = int(3) # int(5)
    # print('now is ' + get_TYPE + ' type.')

  # 6 = vfright_rln_L
  # 7 = vfleft_rln_L
  if get_TYPE == 'RLN_L' or get_TYPE == 'RLN_L_TEST' or get_TYPE == 'RLN_L_VALID':
    mask_vfright[mask_vfright > 0] = int(2) # int(6)
    mask_vfleft[mask_vfleft > 0] = int(3) # int(7)
    # print('now is ' + get_TYPE + ' type.')

  # 8 = vfright_rsln_R
  # 9 = vfleft_rsln_R
  if get_TYPE == 'RSLN_R' or get_TYPE == 'RSLN_R_TEST' or get_TYPE == 'RSLN_R_VALID':
    mask_vfright[mask_vfright > 0] = int(2) # int(8)
    mask_vfleft[mask_vfleft > 0] = int(3) # int(9)
    # print('now is ' + get_TYPE + ' type.')

  # 10 = vfright_rsln_L
  # 11 = vfleft_rsln_L
  if get_TYPE == 'RSLN_L' or get_TYPE == 'RSLN_L_TEST' or get_TYPE == 'RSLN_L_VALID':
    mask_vfright[mask_vfright > 0] = int(2) # int(10)
    mask_vfleft[mask_vfleft > 0] = int(3) # int(11)
    # print('now is ' + get_TYPE + ' type.')

  new_mask = np.concatenate([[mask_frontglottis],[mask_vfright],[mask_vfleft]]).max(axis=0)

  return new_mask

In [22]:
# combine

def combine_all_classes(mask_DIR, get_file, get_TYPE):
  mask_frontglottis = get_frontglottis(mask_DIR, get_file)
  mask_vfleft = get_vfleft(mask_DIR, get_file)
  mask_vfright = get_vfright(mask_DIR, get_file)
  # 0 = ground

  # 1 = frontglottis
  mask_frontglottis[mask_frontglottis > 0] = int(1)

  # 2 = vfright_normal
  # 3 = vfleft_normal
  if get_TYPE == 'NORMAL' or get_TYPE == 'NORMAL_TEST' or get_TYPE == 'NORMAL_VALID':
    mask_vfright[mask_vfright > 0] = int(2)
    mask_vfleft[mask_vfleft > 0] = int(3)
    # print('now is ' + get_TYPE + ' type.')

  # 4 = vfright_rln_R
  # 5 = vfleft_rln_R
  if get_TYPE == 'RLN_R' or get_TYPE == 'RLN_R_TEST' or get_TYPE == 'RLN_R_VALID':
    mask_vfright[mask_vfright > 0] = int(4)
    mask_vfleft[mask_vfleft > 0] = int(5)
    # print('now is ' + get_TYPE + ' type.')

  # 6 = vfright_rln_L
  # 7 = vfleft_rln_L
  if get_TYPE == 'RLN_L' or get_TYPE == 'RLN_L_TEST' or get_TYPE == 'RLN_L_VALID':
    mask_vfright[mask_vfright > 0] = int(6)
    mask_vfleft[mask_vfleft > 0] = int(7)
    # print('now is ' + get_TYPE + ' type.')

  # 8 = vfright_rsln_R
  # 9 = vfleft_rsln_R
  if get_TYPE == 'RSLN_R' or get_TYPE == 'RSLN_R_TEST' or get_TYPE == 'RSLN_R_VALID':
    mask_vfright[mask_vfright > 0] = int(8)
    mask_vfleft[mask_vfleft > 0] = int(9)
    # print('now is ' + get_TYPE + ' type.')

  # 10 = vfright_rsln_L
  # 11 = vfleft_rsln_L
  if get_TYPE == 'RSLN_L' or get_TYPE == 'RSLN_L_TEST' or get_TYPE == 'RSLN_L_VALID':
    mask_vfright[mask_vfright > 0] = int(10)
    mask_vfleft[mask_vfleft > 0] = int(11)
    # print('now is ' + get_TYPE + ' type.')

  new_mask = np.concatenate([[mask_frontglottis],[mask_vfright],[mask_vfleft]]).max(axis=0)

  return new_mask

In [23]:
def get_real_LRid(get_jpg_name):
    if len(get_jpg_name.split('.jpg')[0]) > 3:
        print_name = (get_jpg_name.split('.jpg')[0]).split('_')[-1]
    else:
        print_name = get_jpg_name.split('.jpg')[0]
    return print_name

In [26]:
# counter = 0

# for root_dir_id in DIR_list:
#     for get_jpgs_folder in os.listdir(root_dir_id):
#         if get_jpgs_folder.split('_')[-1] == 'CNN':
#             jpg_name = os.listdir(os.path.join(root_dir_id, get_jpgs_folder))[0]
#             # print(jpg_name)
#             if len(jpg_name.split('.jpg')[0]) > 3:
#                 print_name = (jpg_name.split('.jpg')[0]).split('_')[-1]
#             else:
#                 print_name = jpg_name.split('.jpg')[0]
#             print(print_name, counter)
#             counter += 1

In [24]:
# brightness upper

def brightness_upper(img, GET):
  img_ = img.copy()
  for i in range(0, 1080):
    for j in range(0, 1080):
      if img_[i][j] + GET > np.uint8(255):
        img_[i][j] = np.uint8(255)
      else:
        img_[i][j] = img_[i][j] + GET
  # print(np.unique(img_))
  return img_

In [25]:
# brightness lower

def brightness_lower(img, GET):
  img_ = img.copy()
  for i in range(0, 1080):
    for j in range(0, 1080):
      if img_[i][j] - GET < np.uint8(0):
        img_[i][j] = np.uint8(0)
      else:
        img_[i][j] = img_[i][j] - GET
  # print(np.unique(img_))
  return img_

In [26]:
# single_process

def single_process(get_dir, NEW_DATA_PATH, START_ID, this_TYPE, this_GROUP):
  mask_DIR = os.path.join(get_dir, 'mask')
  jpgs_DIR = os.path.join(mask_DIR, 'jpgs')

  new_train_for_base_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_mask')
  new_train_for_base_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_imgs')
  new_train_for_base_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'train_for_base_imgs_rgb')

  new_train_for_step_mask_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_mask')
  new_train_for_step_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_imgs')
  new_train_for_step_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'train_for_step_imgs_rgb')

  new_valid_mask_DIR = os.path.join(NEW_DATA_PATH, 'valid_mask')
  new_valid_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'valid_imgs')
  new_valid_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'valid_imgs_rgb')

  new_test_mask_DIR = os.path.join(NEW_DATA_PATH, 'test_mask')
  new_test_jpgs_DIR = os.path.join(NEW_DATA_PATH, 'test_imgs')
  new_test_jpgs_rgb_DIR = os.path.join(NEW_DATA_PATH, 'test_imgs_rgb')

  checkpath(new_train_for_base_mask_DIR)
  checkpath(new_train_for_base_jpgs_DIR)
  checkpath(new_train_for_base_jpgs_rgb_DIR)

  checkpath(new_train_for_step_mask_DIR)  
  checkpath(new_train_for_step_jpgs_DIR)
  checkpath(new_train_for_step_jpgs_rgb_DIR)

  checkpath(new_valid_mask_DIR)
  checkpath(new_valid_jpgs_DIR)
  checkpath(new_valid_jpgs_rgb_DIR)

  checkpath(new_test_mask_DIR)
  checkpath(new_test_jpgs_DIR)
  checkpath(new_test_jpgs_rgb_DIR)

  # for found_size in os.listdir(mask_DIR):
  #   if found_size == '488':
  #     size = 488
  #   elif found_size == '1080':
  #     size = 1080

  counter = 0

  for get_file in os.listdir(jpgs_DIR):
    if get_file.split('.')[-1] == 'jpg':

      if this_TYPE.split('_')[-1] == 'TEST':
        ima_mask_DIR = new_test_mask_DIR
        ima_jpgs_DIR = new_test_jpgs_DIR
        ima_jpgs_rgb_DIR = new_test_jpgs_rgb_DIR
      elif this_TYPE.split('_')[-1] == 'VALID':
        ima_mask_DIR = new_valid_mask_DIR
        ima_jpgs_DIR = new_valid_jpgs_DIR
        ima_jpgs_rgb_DIR = new_valid_jpgs_rgb_DIR
      elif  counter % 2 == 0:
        ima_mask_DIR = new_train_for_step_mask_DIR
        ima_jpgs_DIR = new_train_for_step_jpgs_DIR
        ima_jpgs_rgb_DIR = new_train_for_step_jpgs_rgb_DIR
      else:
        ima_mask_DIR = new_train_for_base_mask_DIR
        ima_jpgs_DIR = new_train_for_base_jpgs_DIR
        ima_jpgs_rgb_DIR = new_train_for_base_jpgs_rgb_DIR

      ima_ID = this_TYPE + '_G' + this_GROUP + '_Lid' + str(counter) + '_LRid' + get_real_LRid(get_file) + '_Gid' + str(START_ID) + '.png'

      combined_mask = combine(mask_DIR, get_file.split('.')[0], this_TYPE)
      cv2.imwrite(os.path.join(ima_mask_DIR, ima_ID.replace('.png', '_C4.png')), combined_mask)

      # combined_all_classes_mask = combine_all_classes(mask_DIR, get_file.split('.')[0], this_TYPE)
      # cv2.imwrite(os.path.join(ima_mask_DIR, ima_ID.replace('.png', '_C12.png')), combined_all_classes_mask)

      print('Saving to ' + os.path.join(ima_mask_DIR, (ima_ID.replace('.png', '')).replace(root_dir, '')))

      pair_img = cv2.imread(os.path.join(jpgs_DIR, get_file))

      cv2.imwrite(os.path.join(ima_jpgs_rgb_DIR, ima_ID.replace('.png', '_Bnrgb.png')), pair_img)

      pair_img = cv2.cvtColor(pair_img, cv2.COLOR_BGR2GRAY)

      cv2.imwrite(os.path.join(ima_jpgs_DIR, ima_ID.replace('.png', '_Bn00.png')), pair_img)

      if this_TYPE.split('_')[-1] != 'TEST' and this_TYPE.split('_')[-1] != 'VALID':
        cv2.imwrite(os.path.join(ima_jpgs_DIR, ima_ID.replace('.png', '_Bu30.png')), brightness_upper(pair_img, 30))
        cv2.imwrite(os.path.join(ima_jpgs_DIR, ima_ID.replace('.png', '_Bl30.png')), brightness_lower(pair_img, 30))

      START_ID += 1
      counter += 1

  return START_ID

In [30]:
# # test
# NORMAL_DIR = os.path.join(ROOT_DIR, 'NORMAL')
# mask_DIR = os.path.join(NORMAL_DIR, 'mask')
# get_file = '10.jpg'
# TYPE = 'NORMAL'

# combined_mask = combine(mask_DIR, get_file.split('.')[0], TYPE)
# print(np.unique(combined_mask))

# mask_filename = os.path.join(ROOT_DIR, 'test.png')
# print(mask_filename)
# cv2.imwrite(mask_filename, combined_mask)

# img___ = cv2.imread(mask_filename)
# print(np.unique(img___))

In [28]:
# all data checker

def data_checker(GET_ROOT_DIR):

  print('Now checking in ' + GET_ROOT_DIR.replace(root_root, ''))

  jpgs_counter = 0
  mask_0_counter = 0
  mask_1_counter = 0
  mask_2_counter = 0

  mask_DIR = os.path.join(GET_ROOT_DIR, 'mask')
  jpgs_DIR = os.path.join(mask_DIR, 'jpgs')

  error_list = []
  
  for get_jpgs_file in os.listdir(jpgs_DIR):
    if get_jpgs_file.split('.')[-1] == 'jpg':
      jpgs_counter += 1


    for get_file_name in os.listdir(os.path.join(mask_DIR, 'frontglottis')):
      if get_file_name.replace(get_file_name.split('_')[-1], '') == get_jpgs_file.split('.')[0] + '_mask_frontglottis_':
        mask_0_counter += 1

    if jpgs_counter != mask_0_counter:
      error_list += ['in ' + get_jpgs_file.split('.')[0] + '.json \"frontglottis\" is lost']
      mask_0_counter += 1


    for get_file_name in os.listdir(os.path.join(mask_DIR, 'vfleft')):
      if get_file_name.replace(get_file_name.split('_')[-1], '') == get_jpgs_file.split('.')[0] + '_mask_vfleft_':
        mask_2_counter += 1

    if jpgs_counter != mask_2_counter:
      error_list += ['in ' + get_jpgs_file.split('.')[0] + '.json \"vfleft\" is lost']
      mask_2_counter += 1


    for get_file_name in os.listdir(os.path.join(mask_DIR, 'vfright')):
      if get_file_name.replace(get_file_name.split('_')[-1], '') == get_jpgs_file.split('.')[0] + '_mask_vfright_':
        mask_1_counter += 1
    
    if jpgs_counter != mask_1_counter:
      error_list += ['in ' + get_jpgs_file.split('.')[0] + '.json \"vfright\" is lost']
      mask_1_counter += 1



  # print(f'  {jpgs_counter} / {mask_0_counter} / {mask_1_counter} / {mask_2_counter}')
  # if jpgs_counter == mask_0_counter and jpgs_counter == mask_1_counter and jpgs_counter == mask_2_counter:
  #   print('Check complete in ' + GET_ROOT_DIR + '\n')
  # else:
  #   print('Having some problem in ' + GET_ROOT_DIR + '\n')

    
  # if jpgs_counter != mask_0_counter or jpgs_counter != mask_1_counter or jpgs_counter != mask_2_counter:
  #   print('Having some problem in ' + GET_ROOT_DIR.replace('/home/yasaisen/Desktop/09_research/09_research_main/lab_03/data', ''))
  #   print(f'  {jpgs_counter} / {mask_0_counter} / {mask_1_counter} / {mask_2_counter}\n')
  for last_file_names in error_list:
    print(last_file_names)
    


In [29]:
for root_dir_id in DIR_list:
    data_checker(root_dir_id)

print('\n\nCheck Completed Successfully!!!\n\n')

Now checking in data/TEST_TEST/G_8
Now checking in data/TEST_TEST/G_4
Now checking in data/TEST_TEST/G_2
Now checking in data/TEST_TEST/G_5
Now checking in data/TEST_TEST/G_6
Now checking in data/TEST_TEST/G_7
Now checking in data/TEST_TEST/G_9
Now checking in data/TEST_TEST/G_3
Now checking in data/TEST_TEST/G_1


Check Completed Successfully!!!




In [ ]:
# DIR_list = []

# for get_folder in os.listdir(root_dir):
#   watching_root_dir = os.path.join(root_dir, get_folder)
#   for get_folder_name in os.listdir(watching_root_dir):
#     watching_this_dir = os.path.join(watching_root_dir, get_folder_name)
#     DIR_list += [watching_this_dir]


# print(DIR_list)

In [30]:
# main

ima_ID = 0
for root_dir_id in DIR_list:

  this_type = root_dir_id.split('/')[-2]

  if (root_dir_id.split('/')[-1])[0] == 'G':
    this_group = root_dir_id.split('/')[-1].split('_')[1]
  elif (root_dir_id.split('/')[-1])[0] == '組':
    this_group = root_dir_id.split('/')[-1].split('_')[0].replace('組', '')
  
  print('Now processing in ' + root_dir_id)

  ima_ID = single_process(root_dir_id, new_DATA_PATH, ima_ID, this_type, this_group)

print('\n\nSuccessfully Completed!!!\n\n')


Now processing in /home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/TEST_TEST/G_8
/TEST_TEST/G_8/mask/frontglottis/205_mask_frontglottis_0.jpg
Saving to /home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/../dataset_C_v_3.0.1/test_mask/TEST_TEST_G8_Lid0_LRid205_Gid0
/TEST_TEST/G_8/mask/frontglottis/191_mask_frontglottis_0.jpg
Saving to /home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/../dataset_C_v_3.0.1/test_mask/TEST_TEST_G8_Lid1_LRid191_Gid1
/TEST_TEST/G_8/mask/frontglottis/246_mask_frontglottis_0.jpg
Saving to /home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/../dataset_C_v_3.0.1/test_mask/TEST_TEST_G8_Lid2_LRid246_Gid2
/TEST_TEST/G_8/mask/frontglottis/216_mask_frontglottis_0.jpg
Saving to /home/yasaisen/Desktop/09_research/09_research_main/lab_06/data/../dataset_C_v_3.0.1/test_mask/TEST_TEST_G8_Lid3_LRid216_Gid3
/TEST_TEST/G_8/mask/frontglottis/256_mask_frontglottis_0.jpg
Saving to /home/yasaisen/Desktop/09_research/09_researc

In [ ]:
def outputed_data_checker(get_DATA_PATH):

  outputed_error_list = []

  for get_folder in os.listdir(get_DATA_PATH):
    if get_folder.split('_')[-1] == 'mask':
        mask_DIR = os.path.join(get_DATA_PATH, get_folder)

        for get_mask in os.listdir(mask_DIR):
          img = cv2.imread(os.path.join(mask_DIR, get_mask))
          get_classes_list = np.unique(img).sort()

          if get_mask.split('_')[0] == 'NORMAL':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'R':
            if (get_classes_list == np.array([0, 1, 4, 5])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'L':
            if (get_classes_list == np.array([0, 1, 6, 7])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'R':
            if (get_classes_list == np.array([0, 1, 8, 9])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'L':
            if (get_classes_list == np.array([0, 1, 10, 11])).all():
              print(get_mask)
              outputed_error_list += [get_mask]


  with open('OUTPUTED_ERROR_LIST.txt', 'w') as f:
    for get_error in outputed_error_list:
      print(get_error.replace(root_root, ''))
      f.write(get_error.replace(root_root, '') + '\n')

In [ ]:
def outputed_data_checker(get_DATA_PATH):

  outputed_error_list = []

  for get_folder in os.listdir(get_DATA_PATH):
    if get_folder.split('_')[-1] == 'mask':
        mask_DIR = os.path.join(get_DATA_PATH, get_folder)

        for get_mask in os.listdir(mask_DIR):
          img = cv2.imread(os.path.join(mask_DIR, get_mask))
          get_classes_list = np.unique(img).sort()

          if get_mask.split('_')[0] == 'NORMAL':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'R':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'L':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'R':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]

          if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'L':
            if (get_classes_list == np.array([0, 1, 2, 3])).all():
              print(get_mask)
              outputed_error_list += [get_mask]


  with open('OUTPUTED_ERROR_LIST.txt', 'w') as f:
    for get_error in outputed_error_list:
      print(get_error.replace(root_root, ''))
      f.write(get_error.replace(root_root, '') + '\n')

In [ ]:
def outputed_data_checker(get_DATA_PATH):

  outputed_error_list = []

  for get_folder in os.listdir(get_DATA_PATH):
    if get_folder.split('_')[-1] == 'mask':
        mask_DIR = os.path.join(get_DATA_PATH, get_folder)

        for get_mask in os.listdir(mask_DIR):
          if get_mask.split('_')[-1] == 'C4.png':
            img = cv2.imread(os.path.join(mask_DIR, get_mask))
            get_classes_list = np.unique(img).sort()

            if get_mask.split('_')[0] == 'NORMAL':
                if (get_classes_list == np.array([0, 1, 2, 3])).all():
                    print(get_mask)
                    outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'R':
                if (get_classes_list == np.array([0, 1, 2, 3])).all():
                    print(get_mask)
                    outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'L':
                if (get_classes_list == np.array([0, 1, 2, 3])).all():
                    print(get_mask)
                    outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'R':
                if (get_classes_list == np.array([0, 1, 2, 3])).all():
                    print(get_mask)
                    outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'L':
                if (get_classes_list == np.array([0, 1, 2, 3])).all():
                    print(get_mask)
                    outputed_error_list += [get_mask]
          elif get_mask.split('_')[-1] == 'C12.png':
            img = cv2.imread(os.path.join(mask_DIR, get_mask))
            get_classes_list = np.unique(img).sort()

            if get_mask.split('_')[0] == 'NORMAL':
              if (get_classes_list == np.array([0, 1, 2, 3])).all():
                print(get_mask)
                outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'R':
              if (get_classes_list == np.array([0, 1, 4, 5])).all():
                print(get_mask)
                outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RLN' and get_mask.split('_')[1] == 'L':
              if (get_classes_list == np.array([0, 1, 6, 7])).all():
                print(get_mask)
                outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'R':
              if (get_classes_list == np.array([0, 1, 8, 9])).all():
                print(get_mask)
                outputed_error_list += [get_mask]

            if get_mask.split('_')[0] == 'RSLN' and get_mask.split('_')[1] == 'L':
              if (get_classes_list == np.array([0, 1, 10, 11])).all():
                print(get_mask)
                outputed_error_list += [get_mask]

  with open('OUTPUTED_ERROR_LIST.txt', 'w') as f:
    for get_error in outputed_error_list:
      print(get_error.replace(root_root, ''))
      f.write(get_error.replace(root_root, '') + '\n')

In [ ]:
outputed_data_checker(new_DATA_PATH)

In [ ]:
# mask fit check

mask_DIR = os.path.join(new_DATA_PATH, 'train_1_mask')
jpgs_DIR = os.path.join(new_DATA_PATH, 'train_1_imgs')


for get_file_name in os.listdir(jpgs_DIR):
  print('Now looking ' + get_file_name)

  mask = cv2.imread(os.path.join(mask_DIR, get_file_name))
  img = cv2.imread(os.path.join(jpgs_DIR, get_file_name))

  plt.imshow(img)
  plt.show()

  mask = mask*50
  plt.imshow(mask)
  plt.show()

  res = cv2.addWeighted(img, 0.5, mask, 0.5, 0)
  plt.imshow(res)
  plt.show()

In [ ]:
def prediction_map(get_list, pred_idx):
  imanopredict = []

  for i in range(0,12):
    imanopredict += [int(np.sum(np.asarray(pred_idx)==i))]

  sumR = 0
  sumL = 0

  for i in range(2, 12, 2):
    sumR += imanopredict[i]
    sumL += imanopredict[i+1]

  print('-'*25)
  for i in range(2, 12, 2):
    print(f'{imanopredict[i]/sumR*100:.2f}% = {get_list[i]}')

  print('-'*25)
  for i in range(3, 12, 2):
    print(f'{imanopredict[i]/sumL*100:.2f}% = {get_list[i]}')

  print('-'*25)

In [ ]:
import pandas as pd

In [ ]:
classescsv_path = os.path.join(new_DATA_PATH, "classes.csv")
classescsv = pd.read_csv(classescsv_path)
classes = list(classescsv['_class'])

In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask'

img_ = np.zeros(10)#622810

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  img_ = np.concatenate([img_, img])

prediction_map(classes, img_)

In [ ]:
jpgs_DIR = '/home/yasaisen/Desktop/09_research/09_research_main/lab_03/dataset_C_v_2.4.0/train_1_mask'

img_ = np.zeros(10)#622810

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  img_ = np.concatenate([img_, img])

img_ = np.unique(img_)
print(img_)

In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask/0.png'
jpgs_DIR_ = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask/1039.png'


img = cv2.imread(os.path.join(jpgs_DIR, get_file))
img = np.unique(img)

img_ = cv2.imread(os.path.join(jpgs_DIR_, get_file))
img_ = np.unique(img_)

print(type(img[0]))
print(type(img_[0]))


In [ ]:
jpgs_DIR = '/content/drive/My Drive/09_research_main/lab_01/data/dataset_C_v_1.4.1/mask'

img_ = np.zeros(10)

for get_file in os.listdir(jpgs_DIR):
  img = cv2.imread(os.path.join(jpgs_DIR, get_file))
  img = np.unique(img)
  print(get_file)
  print(img)